# Prepping data

In [23]:
# file imports
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from scipy.optimize import minimize as minimize
from itertools import islice
import time

In [9]:
# splitting data in train test and val set
def data_split(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)  
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)
    return X_train, X_test, X_val, y_train, y_test, y_val

data = pd.read_excel(r'C:\Users\RE-Giorgio\Downloads\dataPoints.xlsx')
X = np.array(data.iloc[:,:2])
y = np.array(data.iloc[:, 2])
X_train, X_test, X_val, y_train, y_test, y_val = data_split(X, y)

# Neural Network parent class

In [145]:
class ShallowNeuralNetwork:
    
    def __init__(self, X, y, N, sigma):

        self.X = X
        self.y = y
        self.N = N
        self.bias = np.random.normal(self.N)
        self.weights1 = np.random.normal(0,1,(self.X.shape[1], self.N))
        self.weights2 = np.random.normal(0,1,(self.N, 1))
        self.output = np.zeros(y.shape[0])
        self.rho = np.random.normal(10e-4,10e-5,1)
        self.sigma = sigma
    
    # needed to apply the norm on the parameters' vectors all togheter for the regularization
    def concatenate(self,l):
        
        l = [np.array(array).reshape(-1,1) for array in l]
        return np.concatenate(l)
    
    # needed to divide the concatenated vectors for the forward propagation
    def separate(self, l):
    
        seclist = [self.X.shape[1]*self.N, self.N, self.N]
        return [list(islice(iter(l), 0, i)) for i in seclist]
    
    def loss(self,params, fixed_params):
        
        weights1, weights2, bias = self.separate(params)
        y, rho, sigma = fixed_params
        return 0.5 * np.mean(np.square((self.act_fun(self.predict(self.X)) - y))) +\
            rho*np.square(np.linalg.norm(self.concatenate([weights1, weights2, bias])))
    
    def mse(self, x, y):
        return 0.5*np.mean(np.square(self.predict(x) - y)) 

## MLP child class

In [146]:
class Mlp(ShallowNeuralNetwork):
    
    def act_fun(self,x):
        return (1-np.exp(-2*x*self.sigma))/(1+np.exp(-2*x*self.sigma))

    def predict(self, x):
        
        hidden_layer = self.act_fun(np.dot(x, self.weights1)) + self.bias
        self.output = np.dot(hidden_layer, self.weights2)
        return self.output
                          
    def optimize(self):
        
        function_args = np.array([self.y, self.rho, self.sigma])
        inits = self.concatenate([self.weights1, self.weights2, self.bias])
        
        start = time.time()
        result =  minimize(self.loss, x0 = inits, method='CG', args = function_args)
        time_elapsed = time.time() - start
        print(result)
        
        self.weights1 = np.array(self.separate(result.x)[0]).reshape((self.X.shape[1], self.N))
        self.weights2 = np.array(self.separate(result.x)[1]).reshape((self.N, 1))
        self.bias = np.array(self.separate(result.x)[2]).reshape(self.N)
        
        func_eval = result.nfev
        grad_eval = result.njev
        iterations = result.nit
        opt_fun = result.fun
        return func_eval, grad_eval, iterations, opt_fun, time_elapsed

## first run

In [148]:
nn = Mlp(X_train, y_train, 10, 1)
func_eval, grad_eval, iterations, opt_fun, time_elapsed = nn.optimize()
print("Training Error :", nn.mse(X_train, y_train))

     fun: 0.6953942762363102
     jac: array([ 2.23517418e-08, -2.98023224e-08,  1.26659870e-07, -5.96046448e-08,
        0.00000000e+00, -2.23517418e-08,  2.98023224e-08, -1.49011612e-08,
        1.49011612e-08, -4.47034836e-08,  7.45058060e-09, -2.98023224e-08,
        7.45058060e-09,  2.98023224e-08,  1.49011612e-08, -2.23517418e-08,
        7.45058060e-09,  2.23517418e-08,  2.23517418e-08, -1.49011612e-08,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00])
 message: 'Optimization terminated successfully.'
    nfev: 858
     nit: 5
    njev: 26
  status: 0
 success: True
       x: array([ 3.33708159e-06, -5.81336332e-06,  1.93344472e-05, -9.95628536e-06,
       -6.20114647e-07, -4.29822550e-06,  4.59878652e-06, -3.13316252e-06,
        1.68317594e-06, -7.14026061e-06,  1.29653849e-06, -1.51407286e-05,
        1.01954028e-06,  1

In [150]:
nn.predict(X_test)
nn.mse(X_test, y_test)

1.2219066124006976